In [648]:
import pandas as pd
df = pd.read_parquet("data/vogue_data_cd.parquet")
df = df[df["fashion_house"] != "Fashion East"].reset_index(drop=True)
df = df[df["fashion_house"] != "Theory"].reset_index(drop=True)
df = df[df.groupby("fashion_house")["fashion_house"].transform("count") >= 10]
#df = df.drop(columns=[col for col in ["designer_name"] if col in df.columns])

problem_rows = df[df["designer_name"].apply(lambda x: len(x) ==0)]

In [645]:
df = df.drop(columns=[col for col in ["designer_names","ner_person_names"] if col in df.columns])

In [595]:
df[
    (df["fashion_house"] == "Theory") 
    # &
    # (df["designer_name"].apply(lambda x: "Vittorio and Laura Moltedo" in list(x)))
]

,fashion_house,show,URL,image_urls,location,season,year,category,image_urls_sample,description,editor,publish_date,ner_person_names,designer_names,designer_name
